In [1]:
"""
Apply disk filter
"""

imgs = [
    '/mnt/d/gwork/osm2lulc_imgcls/rs_paper/results_randomforest/lsb_f0_500k_4x3_05kt_v2.tif',
    '/mnt/d/gwork/osm2lulc_imgcls/rs_paper/results_randomforest/lsb_f1_500k_4x3_05kt_v2.tif',
    '/mnt/d/gwork/osm2lulc_imgcls/rs_paper/results_randomforest/lsb_f2_500k_4x3_05kt_v2.tif',
    #'/mnt/d/gwork/osm2lulc_imgcls/rs_paper/results_randomforest/pnse_f0_500k_4x3_05kt.tif',
    #'/mnt/d/gwork/osm2lulc_imgcls/rs_paper/results_randomforest/pnse_f1_500k_4x3_05kt.tif',
    #'/mnt/d/gwork/osm2lulc_imgcls/rs_paper/results_randomforest/pnse_f2_500k_4x3_05kt.tif',
]

radius = [4, 5, 7, 10, 12]

outfolder = '/mnt/d/gwork/osm2lulc_imgcls/rs_paper/results_randomforest/disk_genze/'

In [2]:
import os
from osgeo import gdal
from skimage.filters.rank import majority
from skimage.morphology import disk
from glass.g.it.rst import obj_to_rst
from glass.pys.oss import fprop
import numpy as np

In [3]:
# Create Disks
disks = [disk(r) for r in radius]

# Get new images and save them in new files
for i in imgs:
    # Get filename
    fn = fprop(i, 'fn')
    
    # Open image
    _i = gdal.Open(i, gdal.GA_ReadOnly)
    
    # Get nodata value
    nd = _i.GetRasterBand(1).GetNoDataValue()
    
    # Read data as Array
    num = _i.ReadAsArray()
        
    for r in range(len(radius)):
        # Get new image - majority class
        ni = majority(num, disks[r])
        
        # Place nodata
        np.place(ni, num==nd, nd)
        
        # Save result
        obj_to_rst(ni, os.path.join(
            outfolder, fn + '_d' + str(radius[r]) + '.tif'
        ), _i, noData=nd)